# Advent of Code 2024 Day 17 (Chronospatial Computer)

## Part 1

In [1]:
from dataclasses import dataclass
from enum import auto, Enum
import copy
import math

In [2]:
OperandType = Enum("OperandType", "COMBO LITERAL")

class Opcode(Enum):
    ADV = 0
    BXL = auto()
    BST = auto()
    JNZ = auto()
    BXC = auto()
    OUT = auto()
    BDV = auto()
    CDV = auto()

    @property
    def operand_type(self) -> OperandType | None:
        match self:
            case self.ADV | self.BST | self.OUT | self.BDV | self.CDV:
                operand_type = OperandType.COMBO
            case self.BXL | self.JNZ:
                operand_type = OperandType.LITERAL
            case _:
                operand_type = None

        return operand_type

@dataclass(slots=True, frozen=True)
class Operand:
    value: int
    category: OperandType

@dataclass(slots=True)
class Pointer:
    position: int = 0
    is_on_opcode: bool = True

    def move(self):
        self.position += 1
        self.is_on_opcode = False if self.is_on_opcode else True

    def jump(self, new_pos):
        self.position = new_pos
        self.is_on_opcode = True

    def reset(self):
        self.position = 0
        self.is_on_opcode = True

@dataclass(slots=True, frozen=True)
class Instruction:
    opcode: Opcode
    operand: Operand

    def operate(self, registers: dict):
        operand_val = self.get_operand_value(registers)
                        
        match self.opcode:
            case Opcode.ADV:
                registers["A"] = math.trunc(registers["A"] / (2**operand_val))
            case Opcode.BDV:
                registers["B"] = math.trunc(registers["A"] / (2**operand_val))
            case Opcode.CDV:
                registers["C"] = math.trunc(registers["A"] / (2**operand_val))
            case Opcode.BXL:
                registers["B"] = registers["B"] ^ operand_val
            case Opcode.BST:
                registers["B"] = operand_val % 8
            case Opcode.BXC:
                registers["B"] = registers["B"] ^ registers["C"]
            case _:
                result = None

    def get_output(self, registers):
        output = self.get_operand_value(registers) % 8
        return output

    def get_operand_value(self, registers):
        if self.operand.category == OperandType.COMBO:
            match self.operand.value:
                case n if all([0 <= n, n <= 3]):
                    operand_result = self.operand.value
                case 4:
                    operand_result = registers["A"]
                case 5:
                    operand_result = registers["B"]
                case 6:
                    operand_result = registers["C"]
                case _:
                    raise ValueError("invalid operand value [0-6]")
        elif self.operand.category == OperandType.LITERAL:
            if any([self.operand.value < 0, self.operand.value > 6]):
                raise ValueError("invalid operand value [0-6]")
            operand_result = self.operand.value
        else:
            operand_result = None

        return operand_result

def run_program(codes: list[str], registers: dict[str, int], pointer: Pointer, matching: bool = False):
    output = []
    
    while not program_is_finished(codes, pointer):
        if pointer.is_on_opcode:
            instruction = None
            opcode = Opcode(int(codes[pointer.position]))
        else:
            operand = Operand(int(codes[pointer.position]), opcode.operand_type)
            instruction = Instruction(opcode, operand)
            
            if instruction.opcode == Opcode.JNZ and registers["A"] != 0:
                pointer.jump(instruction.operand.value)
                continue
            elif instruction.opcode == Opcode.OUT:
                output.append(instruction.get_output(registers))
                if matching and len(output) > len(codes):
                    break
            else:
                instruction.operate(registers)
                
        pointer.move()

    return output

def program_is_finished(codes: list[str], pointer: Pointer):
    return pointer.position > len(codes) - 1

In [3]:
pointer = Pointer()
registers = dict()
output = []

with open("data/instructions.txt", "r") as file:
    content = [line.strip() for line in file.readlines()]
    
    for row in content:
        if "Register" in row:
            register_row = row.split()
            key = register_row[-2].replace(":", "")
            value = register_row[-1]
            registers[key] = int(value)
        elif "Program" in row:
            codes = row.replace("Program: ", "").split(",")
            output = run_program(codes, registers, pointer)

                
p1_output = ",".join([str(out) for out in output])
p1_output

'6,7,5,2,1,3,5,1,7'

## Part 2

#### This is a big number. The test set works, but the real answer seems to be prohibitively large. It looks like there is some sort of correlation between the A register and the length of the output list as well as a deterministic pattern to the output values themselves. I may come back for part 2 later.

In [4]:
def output_matches_program(output, program):
    return (
        ",".join([str(o) for o in output]) == ",".join([str(p) for p in program])
    ) if output else False

In [5]:
pointer = Pointer()
real_registers = {"A": 0, "B": 0, "C": 0}
program = []
output = []

with open("data/test.txt", "r") as file:
    program = [line.strip().replace("Program: ", "").split(",") for line in file.readlines() if "Program" in line][0]

while not output_matches_program(output, program):
    regs_copy = copy.deepcopy(real_registers)
    pointer.reset()
    output = run_program(program, regs_copy, pointer, matching=True)
    real_registers["A"] += 1

real_registers["A"]

117441

In [6]:
with open("answer.txt", "w") as file:
    file.writelines([p1_output])